
# Benchmark Runtime and Memory Overview

This notebook loads QuASAr benchmark runs from the SQLite database produced by the
benchmark sweep helpers and visualises runtime and peak memory consumption trends
as a function of qubit count and circuit depth.



## Setup

Update `DB_PATH` below if your benchmark database lives somewhere else. The
default points at `benchmarks.db` in the repository root, which is where the
benchmark sweep helpers persist their results by default.


In [ ]:

from __future__ import annotations

from pathlib import Path
import sqlite3

import matplotlib.pyplot as plt

try:
    import pandas as pd
except ImportError as exc:  # pragma: no cover - notebook guard
    raise ImportError(
        "This notebook requires pandas. Install it with `pip install pandas`."
    ) from exc


In [ ]:

DB_PATH = Path("benchmarks.db")
if not DB_PATH.exists():  # pragma: no cover - user environment check
    raise FileNotFoundError(
        f"Could not find {DB_PATH}. Update `DB_PATH` to the location of your benchmark database."
    )

plt.style.use("ggplot")


In [ ]:

def load_quasar_runs(db_path: Path) -> pd.DataFrame:
    """Load QuASAr benchmark runs that have runtime or memory statistics."""
    columns = [
        "id",
        "created_at",
        "family",
        "varying",
        "varying_value",
        "num_qubits",
        "depth",
        "exec_wall_s",
        "peak_rss_bytes",
    ]
    query = (
        "SELECT {cols} FROM quasar_runs "
        "WHERE exec_wall_s IS NOT NULL OR peak_rss_bytes IS NOT NULL"
    ).format(cols=", ".join(columns))
    with sqlite3.connect(db_path) as conn:
        frame = pd.read_sql_query(query, conn, parse_dates=["created_at"])
    frame["peak_rss_gib"] = frame["peak_rss_bytes"].astype(float) / 1024 ** 3
    return frame


def summarise_by_metric(
    frame: pd.DataFrame, *, axis: str, metric: str, agg: str = "mean"
) -> pd.DataFrame:
    """Aggregate the selected metric over the requested axis and circuit family."""
    if axis not in {"num_qubits", "depth"}:
        raise ValueError("axis must be either 'num_qubits' or 'depth'")
    grouped = (
        frame.dropna(subset=[metric])
        .groupby(["family", axis], dropna=False)
        .agg({metric: agg})
        .rename(columns={metric: f"{metric}_{agg}"})
        .reset_index()
    )
    return grouped


def plot_metric(grouped: pd.DataFrame, *, axis: str, metric_label: str, ylabel: str) -> None:
    fig, ax = plt.subplots(figsize=(8, 5))
    for family, family_frame in grouped.groupby("family"):
        family_frame = family_frame.sort_values(axis)
        ax.plot(
            family_frame[axis],
            family_frame.iloc[:, 2],
            marker="o",
            label=family,
        )
    ax.set_xlabel(axis.replace("_", " ").title())
    ax.set_ylabel(ylabel)
    ax.set_title(f"{metric_label} vs. {axis.replace('_', ' ').title()}")
    ax.legend(title="Circuit family", loc="best")
    ax.grid(True)
    plt.tight_layout()



## Load benchmark data

The cell below inspects the database and provides a preview of the records that
include runtime or memory statistics.


In [ ]:

quasar_runs = load_quasar_runs(DB_PATH)
quasar_runs.head()



## Runtime and memory trends vs. qubit count

The following plots compute the mean runtime and peak resident-set size for
runs grouped by circuit family and qubit count.


In [ ]:

runtime_vs_qubits = summarise_by_metric(quasar_runs, axis="num_qubits", metric="exec_wall_s")
memory_vs_qubits = summarise_by_metric(quasar_runs, axis="num_qubits", metric="peak_rss_gib")


In [ ]:

plot_metric(
    runtime_vs_qubits,
    axis="num_qubits",
    metric_label="Runtime (s)",
    ylabel="Mean execution wall-clock time (s)",
)


In [ ]:

plot_metric(
    memory_vs_qubits,
    axis="num_qubits",
    metric_label="Peak memory (GiB)",
    ylabel="Mean peak RSS (GiB)",
)



## Runtime and memory trends vs. circuit depth

Similarly, we can look at the same metrics grouped by circuit depth.


In [ ]:

runtime_vs_depth = summarise_by_metric(quasar_runs, axis="depth", metric="exec_wall_s")
memory_vs_depth = summarise_by_metric(quasar_runs, axis="depth", metric="peak_rss_gib")


In [ ]:

plot_metric(
    runtime_vs_depth,
    axis="depth",
    metric_label="Runtime (s)",
    ylabel="Mean execution wall-clock time (s)",
)


In [ ]:

plot_metric(
    memory_vs_depth,
    axis="depth",
    metric_label="Peak memory (GiB)",
    ylabel="Mean peak RSS (GiB)",
)
